In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR, SVR
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('readings/matrix_multiplication_final.csv')
df.head()

,Dimensions,Blocks,Threads,Time,Speedup,[CUDA memcpy DtoH] Time,[CUDA memcpy DtoH] Avg,[CUDA memcpy DtoH] Min,[CUDA memcpy DtoH] Max,[CUDA memcpy HtoD] Time,[CUDA memcpy HtoD] Avg,[CUDA memcpy HtoD] Min,[CUDA memcpy HtoD] Max,cudaMemcpy Time,cudaMemcpy Avg,cudaMemcpy Min,cudaMemcpy Max,cudaMalloc Time,cudaMalloc Avg,cudaMalloc Min,cudaMalloc Max,cudaFree Time,cudaFree Avg,cudaFree Min,cudaFree Max,Device Name,Compute Capability,Total Global Memory,Shared Memory per Block,Registers per Block,Warp Size,Maximum Threads per Block,Thread Dimension Z,Thread Dimension Y,Thread Dimension X,Grid Size Z,Grid Size Y,Grid Size X,Clock Rate,Total Constant Memory,Multiprocessor Count,integrated,Asynchronous Engine Count,Memory Bus Width,Memory Clock Rate,L2 Cache Size,Maximum Threads per Multiprocessor,Concurrent Kernels,float,unsigned_int,int,num_blocks,num_grid,threads,cudaMalloc,cudaMemcpy(,cudaFree
0,256,128,1,0.06,1.000000,0.00124,0.00124,0.00124,0.00124,0.001204,0.000401,0.000345,0.00044,0.002457,0.000614,0.000349,0.001246,0.002403,0.000801,0.000008,0.002387,0.000688,0.000229,0.000023,0.000334,NVIDIA GeForce GTX TITAN Black,3.5,6229696.0,49152.0,65536.0,32.0,1024.0,1024.0,1024.0,64.0,65535.0,65535.0,2.147484e+09,980000.0,65536.0,15.0,0.0,1.0,384.0,3500000.0,1572864.0,2048.0,1.0,28,0,27,1,0,10,3,4,3
1,256,64,2,0.07,1.166667,0.00124,0.00124,0.00124,0.00124,0.001204,0.000401,0.000345,0.00044,0.002457,0.000614,0.000349,0.001246,0.002403,0.000801,0.000008,0.002387,0.000688,0.000229,0.000023,0.000334,NVIDIA GeForce GTX TITAN Black,3.5,6229696.0,49152.0,65536.0,32.0,1024.0,1024.0,1024.0,64.0,65535.0,65535.0,2.147484e+09,980000.0,65536.0,15.0,0.0,1.0,384.0,3500000.0,1572864.0,2048.0,1.0,28,0,27,1,0,10,3,4,3
2,256,32,3,0.06,1.000000,0.00124,0.00124,0.00124,0.00124,0.001204,0.000401,0.000345,0.00044,0.002457,0.000614,0.000349,0.001246,0.002403,0.000801,0.000008,0.002387,0.000688,0.000229,0.000023,0.000334,NVIDIA GeForce GTX TITAN Black,3.5,6229696.0,49152.0,65536.0,32.0,1024.0,1024.0,1024.0,64.0,65535.0,65535.0,2.147484e+09,980000.0,65536.0,15.0,0.0,1.0,384.0,3500000.0,1572864.0,2048.0,1.0,28,0,27,1,0,10,3,4,3
3,256,16,4,0.07,1.166667,0.00124,0.00124,0.00124,0.00124,0.001204,0.000401,0.000345,0.00044,0.002457,0.000614,0.000349,0.001246,0.002403,0.000801,0.000008,0.002387,0.000688,0.000229,0.000023,0.000334,NVIDIA GeForce GTX TITAN Black,3.5,6229696.0,49152.0,65536.0,32.0,1024.0,1024.0,1024.0,64.0,65535.0,65535.0,2.147484e+09,980000.0,65536.0,15.0,0.0,1.0,384.0,3500000.0,1572864.0,2048.0,1.0,28,0,27,1,0,10,3,4,3
4,256,8,5,0.07,1.166667,0.00124,0.00124,0.00124,0.00124,0.001204,0.000401,0.000345,0.00044,0.002457,0.000614,0.000349,0.001246,0.002403,0.000801,0.000008,0.002387,0.000688,0.000229,0.000023,0.000334,NVIDIA GeForce GTX TITAN Black,3.5,6229696.0,49152.0,65536.0,32.0,1024.0,1024.0,1024.0,64.0,65535.0,65535.0,2.147484e+09,980000.0,65536.0,15.0,0.0,1.0,384.0,3500000.0,1572864.0,2048.0,1.0,28,0,27,1,0,10,3,4,3


In [3]:
df.columns

Index(['Dimensions', 'Blocks', 'Threads', 'Time', 'Speedup',
       '[CUDA memcpy DtoH] Time', '[CUDA memcpy DtoH] Avg',
       '[CUDA memcpy DtoH] Min', '[CUDA memcpy DtoH] Max',
       '[CUDA memcpy HtoD] Time', '[CUDA memcpy HtoD] Avg',
       '[CUDA memcpy HtoD] Min', '[CUDA memcpy HtoD] Max', 'cudaMemcpy Time',
       'cudaMemcpy Avg', 'cudaMemcpy Min', 'cudaMemcpy Max', 'cudaMalloc Time',
       'cudaMalloc Avg', 'cudaMalloc Min', 'cudaMalloc Max', 'cudaFree Time',
       'cudaFree Avg', 'cudaFree Min', 'cudaFree Max', 'Device Name',
       'Compute Capability', 'Total Global Memory', 'Shared Memory per Block',
       'Registers per Block', 'Warp Size', 'Maximum Threads per Block',
       'Thread Dimension Z', 'Thread Dimension Y', 'Thread Dimension X',
       'Grid Size Z', 'Grid Size Y', 'Grid Size X', 'Clock Rate',
       'Total Constant Memory', 'Multiprocessor Count', 'integrated',
       'Asynchronous Engine Count', 'Memory Bus Width', 'Memory Clock Rate',
       'L2 Cache 

In [ ]:
y = df['Speedup'].values
df.drop(['Speedup'], axis = 1, inplace = True)

In [ ]:
df.info()

In [ ]:
df['Device Name'].unique().tolist()

In [ ]:
sns.distplot(np.log1p(y))

In [ ]:
X = df.drop(['Device Name'], axis = 1)

In [ ]:
X.columns

In [ ]:
sc = RobustScaler()
X_scaled = sc.fit_transform(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size = 0.25, random_state = 42)

In [ ]:
def evaluate_model(model, X_train, y_train, X_val, y_val, metric='R2'):
    model.fit(X_train, y_train)
    if metric == 'RMSE':
        y_pred_train = model.predict(X_train)
        print('Training RMSE score = {}'.format(np.sqrt(mean_squared_error(y_train, y_pred_train))))
        y_pred = model.predict(X_val)
        print('Validation RMSE score = {}'.format(np.sqrt(mean_squared_error(y_val, y_pred))))
    else:
        print('Training R2 score = {}'.format(model.score(X_train, y_train)))
        print('Validation R2 score = {}'.format(model.score(X_val, y_val)))

In [ ]:
lr = LinearRegression()
evaluate_model(lr, X_train, y_train, X_val, y_val, 'RMSE')

In [ ]:
lasso = Lasso()
evaluate_model(lasso, X_train, y_train, X_val, y_val, 'RMSE')

In [ ]:
ridge = Ridge()
evaluate_model(ridge, X_train, y_train, X_val, y_val, 'RMSE')

In [ ]:
svr = SVR()
evaluate_model(svr, X_train, y_train, X_val, y_val, 'RMSE')

In [ ]:
lsvr = LinearSVR()
evaluate_model(lsvr, X_train, y_train, X_val, y_val, 'RMSE')